# "TensorFlow for Mac OS Quick Start"
> "This notebook shows a very quick example of how to run standard TensorFow 2.0 code in the Tensorflow for MacOS environment. The code in this notebook is basically a port of the TensorFlow 2 quickstart for beginners from https://www.tensorflow.org/tutorials/quickstart/beginner."

- toc: true
- branch: master
- badges: true
- comments: true
- author: tensorflow.org, Ed Martinez
- categories: [tensorflow, MacOS, keras]


# TensorFlow for Mac OS Quick Start

This notebook shows a very quick example of how to run standard TensorFow 2.0 code in the Tensorflow for MacOS environment. The code in this notebook is basically a port of the *TensorFlow 2 quickstart for beginners* from `https://www.tensorflow.org/tutorials/quickstart/beginner`.

The code is mostly unchaged from the original tutorial. I have added in a couple of places comments and/or print statements to check and see what is going on.

The goal is to (hopefully) demonstrate that you should be able (for the most part) take regular Tensorflow 2.0 code and run it under the Mac OS implementation.

## References

* Tensorflow 2 Quickstart. https://www.tensorflow.org/tutorials/quickstart/beginner

## Setup environment

In [13]:
import tensorflow as tf

In [14]:
print(tf.__version__)

2.4.0-rc0


This example will use a dataset that is part of TF 2.0 MNIST. Load the dataset and convert the samples from integers to floating point numbers. 

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11493376/11490434 [==============================] - 1s 0us/step


The example uses a very simmple mode - The Keras `sequential` model. Build the tf.keras.Sequential model by stacking layers. Choose an optimizer and loss function for training:

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "logits" or "log-odds" scores, one for each class.

In [4]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.3319815 , -0.23297991,  0.08580068,  0.6656826 , -0.14421695,
        -0.53031254,  0.78752965,  0.5585072 ,  0.15274768,  0.6479464 ]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class:

In [5]:
tf.nn.softmax(predictions).numpy()

array([[0.10174611, 0.05783066, 0.07954316, 0.14205036, 0.0631986 ,
        0.04295642, 0.16045743, 0.12761351, 0.08505063, 0.13955314]],
      dtype=float32)


The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a True index and returns a scalar loss for each example.

In [6]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class: It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to -tf.log(1/10) ~= 2.3.

In [7]:
loss_fn(y_train[:1], predictions).numpy()

3.1475687

Compile the model

In [8]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

Now fit the model

In [9]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.5005 - accuracy: 0.8489
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1974 - accuracy: 0.9408
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1597 - accuracy: 0.9523
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1359 - accuracy: 0.9577
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1263 - accuracy: 0.9608


The `Model.evaluate` method checks the models performance, usually on a "Validation-set" or "Test-set".

In [10]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.0974 - accuracy: 0.9706


[0.09736651927232742, 0.9706000089645386]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the TensorFlow tutorials.

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [11]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [12]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[3.04166275e-10, 1.44320328e-11, 3.49433162e-06, 1.35840196e-03,
        1.29123058e-12, 1.50930397e-08, 2.67769429e-16, 9.98633206e-01,
        3.44230671e-06, 1.45542526e-06],
       [5.36175207e-07, 3.06448856e-05, 9.99893904e-01, 5.93089026e-05,
        3.26807116e-17, 2.62504585e-07, 1.43621257e-08, 7.79763827e-14,
        1.52049215e-05, 7.31219664e-13],
       [5.85777578e-08, 9.99133885e-01, 6.41653896e-05, 3.23419226e-05,
        1.75682599e-05, 3.31187880e-06, 3.60298395e-06, 4.57659306e-04,
        2.86272872e-04, 1.06325695e-06],
       [9.99991655e-01, 1.56686377e-13, 5.21514312e-06, 3.77243801e-08,
        2.72544834e-12, 5.37931612e-07, 4.18074734e-07, 1.92167181e-06,
        4.93313890e-10, 1.81977938e-07],
       [6.88822911e-06, 8.98235826e-12, 5.17887929e-05, 3.34327638e-07,
        9.75849807e-01, 8.88953906e-08, 9.21286812e-07, 5.88996976e-04,
        5.19101104e-06, 2.34958865e-02]], dtype=float32)>